# Modifying Grids

## Resampling `Grids`

If you want to resample a grid after instantiation, you can apply the `interp_spectra` method:

In [ ]:
import numpy as np
from unyt import angstrom

from synthesizer import Grid

grid = Grid("test_grid.hdf5", grid_dir="../../../tests/test_grid")

# Define a new set of wavelengths
new_lams = np.logspace(2, 5, 10000) * angstrom

print("The old grid had dimensions:", grid.spectra["incident"].shape)

# Get the grid interpolated onto the new wavelength array
grid.interp_spectra(new_lam=new_lams)

print("The interpolated grid has dimensions:", grid.spectra["incident"].shape)

Note that this will overwrite the spectra and wavelengths read from the file *in place*.
To get back to the original arrays, a separate `Grid` can be instantiated without the modified wavelength array.

## Collapsing `Grids`

While most of the models within Synthesizer are capable of handling higher dimensionality grids (i.e. grids with more dimensions than `age` and `metallicity`), other workflows might require some method to reduce the dimensionality. 

This functionality is provided via the `collapse()` method, which collapses the grid over a specified axis.
There are three ways to actually collapse the grid, specified by the `method` keyword argument:

- `marginalize` over the entire axis. This is useful if you don't know anything about this parameter, and just want to adopt the average over it. You can specify the function used to marginalize with the keyword argument `marginalize_function`; the default is `np.average`. 
- Pick the value `nearest` to a specified value. If you know the value of the parameter you want to use, you can collapse the grid by picking the value closest to your specified value. For this, you need to specify the `value` keyword argument. 
- `interpolate` to a specified value. Similar to `nearest`, but with a linear interpolation to your specified `value`. This is useful in workflows where you can't adopt a discrete value, but be warned that interpolating over a coarse grid can give unrealistic results. You can apply a transformation to the axis before interpolating, e.g. to interpolate in log-space rather than linear space, with the keyword argument `pre_interp_function`. 

For example, here we collapse the grid over the metallicity axis:

In [ ]:
print("The old grid had dimensions:", grid.spectra["incident"].shape)
print("and axes:", ", ".join(grid.axes))

# Collapse the grid to a single metallicity value
grid.collapse("metallicities", value=0.03, method="nearest")

print("The collapsed grid has dimensions:", grid.spectra["incident"].shape)
print("and axes:", ", ".join(grid.axes))

Note that `collapse()` will overwrite the `Grid` _in place_. You can restore the grid to its original dimensionality by re-loading from the HDF5 file:

In [ ]:
# Re-load the original grid
grid = Grid("test_grid")

## Converting a `Grid` into an `Sed`

Any of the spectra arrays stored within a `Grid` can be returned as `Sed` objects (see the `Sed` [docs](../emissions/emission_objects/sed_example.ipynb)). This enables all of the analysis methods provide on an Sed to be used on the whole spectra grid. To do this we simply call `get_sed` with the spectra type we want to extract, and then use any of the included methods.

In [ ]:
# Get the sed object
sed = grid.get_sed(spectra_type="incident")

# Measure the balmer break for all spectra in the grid (limiting the output)
sed.measure_balmer_break()[5:10, 5:10]

## Working with flattened grids

Sometimes it's useful to work with flattened (i.e. one dimensional) versions of a grid of spectra, photometry etc. To facilitate this the `get_flattened_axes_values` method on `grid` can be used to get the flattened axes values.

In [ ]:
# Get the flattened version of the axes
flattened_axes_values = grid.get_flattened_axes_values()